<a href="https://colab.research.google.com/github/Nid989/Langchain-Overview/blob/main/langchain_prompt_%26_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain openai python-dotenv --quiet

In [2]:
import os
import dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import  StrOutputParser
from langchain.output_parsers.openai_functions import (
    JsonOutputFunctionsParser,
    JsonKeyOutputFunctionsParser
)
from langchain.schema.runnable import RunnableMap, RunnablePassthrough

dotenv.load_dotenv("./.env.txt")

True

`PromptTemplate` + `LLM`

In [3]:
# PromptTemplate
prompt = ChatPromptTemplate.from_template("tell me a joke about {foo}")

# LLM
model = ChatOpenAI(openai_api_key=os.environ.get("OPENAI_API_KEY"))

# Chain: that takes user input, adds it to a prompt, passes it to a model,
# and returns the raw model output.
chain = prompt | model

In [4]:
chain.invoke({"foo": "bears"})

AIMessage(content="Why don't bears wear shoes?\n\nBecause they have bear feet!")

In [5]:
# Attaching stop sequences
chain = prompt | model.bind(stop=["\n"])

In [7]:
chain.invoke({"foo": "bears"})

AIMessage(content="Why don't bears wear shoes?")

In [8]:
# Attaching function call information
functions = [
    {
        "name": "joke",
        "description": "A joke",
        "parameters": {
            "type": "object",
            "properties": {
                "setup": {
                    "type": "string",
                    "description": "The setup for the joke"
                },
                "punchline": {
                    "type": "string",
                    "description": "The punchline for the joke"
                }
            },
            "required": ["setup", "punchline"]
        }
    }
]
chain = prompt | model.bind(function_call = {"name": "joke"}, functions=functions)

In [9]:
chain.invoke({"foo": "bears"}, config={})

AIMessage(content='', additional_kwargs={'function_call': {'name': 'joke', 'arguments': '{\n  "setup": "Why don\'t bears like fast food?",\n  "punchline": "Because they can\'t catch it!"\n}'}})

`PromptTemplate` + `LLM` + `OutputParser`

In [10]:
# Addition of an output parser to easily transform the raw LLM/ChatModel output
# into a more workable format.
chain = prompt | model | StrOutputParser()

In [11]:
chain.invoke({"foo": "bears"})

"Why don't bears wear shoes? \n\nBecause they already have bear feet!"

In [12]:
# Functions Output Parser
chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonOutputFunctionsParser()
)

In [13]:
chain.invoke({"foo": "bears"})

{'setup': "Why don't bears like fast food?",
 'punchline': "Because they can't catch it!"}

In [14]:
chain = (
    prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

In [15]:
chain.invoke({"foo": "bears"})

"Why don't bears wear shoes?"

`Simplifying Input`

In [16]:
# Takes care of creating the prompt input dict, thus making invocation even
# simpler
map_ = RunnableMap(foo=RunnablePassthrough())

chain = (
    map_
    | prompt
    | model.bind(function_call={"name": "joke"}, functions=functions)
    | JsonKeyOutputFunctionsParser(key_name="setup")
)

# OR instead of using 2 Runnable
# chain = (
#     {"foo": RunnablePassthrough()}
#     | prompt
#     | model.bind(function_call={"name": "joke"}, functions=functions)
#     | JsonKeyOutputFunctionsParser(key_name="setup")
# )

In [17]:
chain.invoke("bears")

"Why don't bears like fast food?"